In [76]:
# imports
import requests
import pandas as pd
import json

In [77]:
#@title Preencha aqui com as informações do cliente requisitante

#@markdown Informe todos os campos a seguir:
Name = "Teste" #@param {type:"string"}
Gender = "Male" #@param ["Selecione", "Male", "Joint", "Sex Not Available", "Female"]
Region = "south" #@param ["Selecione", "North", "south", "central", "North-East"]
income = 10000.0 #@param {type:"number"}
property_value = 5000.0 #@param {type:"number"}
LTV = 100.0 #@param {type:"number"}
business_or_commercial = "nob/c" #@param ["Selecione", "nob/c", "b/c"]
lump_sum_payment = "lpsm" #@param ["Selecione", "not_lpsm", "lpsm"]
loan_amount = 50000 #@param {type:"integer"}
loan_limit =  "cf" #@param ["Selecione", "cf", "ncf"]
loan_type = "type2" #@param ["Selecione", "type1", "type2", "type3"]
approv_in_adv = "nopre" #@param ["Selecione", "nopre", "pre"]
Credit_Worthiness = "l2" #@param ["Selecione", "l1", "l2"]
credit_type = "CRIF" #@param ["Selecione", "CIB", "CRIF", "EXP", "EQUI"]
co_applicant_credit_type = "CRIF" #@param ["Selecione", "CIB", "CRIF", "EXP", "EQUI"]
Neg_ammortization = "not_neg" #@param ["Selecione", "not_neg", "neg_amm"]
submission_of_application = "to_inst" #@param ["Selecione", "to_inst", "not_inst"]

In [78]:
colunas = ['loan_amount', 'property_value', 'income', 'LTV', 'loan_limit', 'Credit_Worthiness',
            'Neg_ammortization', 'Gender', 'loan_type', 'credit_type', 'Region', 'approv_in_adv',
             'business_or_commercial', 'lump_sum_payment', 'co_applicant_credit_type','submission_of_application']

dados = [[loan_amount, property_value, income, LTV, loan_limit, Credit_Worthiness,
            Neg_ammortization, Gender, loan_type, credit_type, Region, approv_in_adv,
             business_or_commercial, lump_sum_payment, co_applicant_credit_type,submission_of_application]]

clientes = pd.DataFrame(data = dados, columns = colunas)

In [79]:
clientes

,loan_amount,property_value,income,LTV,loan_limit,Credit_Worthiness,Neg_ammortization,Gender,loan_type,credit_type,Region,approv_in_adv,business_or_commercial,lump_sum_payment,co_applicant_credit_type,submission_of_application
0,50000,5000.0,10000.0,100.0,cf,l2,not_neg,Male,type2,CRIF,south,nopre,nob/c,lpsm,CRIF,to_inst


# Predição

Realizando apredição com base nos dados inseridos

In [80]:
my_server_ip = "127.0.0.1"
my_server_port = "8080"

url = "http://{}:{}/predict".format(my_server_ip, my_server_port)
headers = {'Content-Type': 'application/json'}
conteudo = clientes.to_json()

In [81]:
response01 = requests.request("POST", url, headers=headers, data=conteudo)
respostas01 = json.loads(response01.text.encode('utf8').decode())
respostas01

{'prediction': '0'}

In [82]:
cluster = pd.Series(respostas01['prediction'], name="Cluster")
cluster = cluster.replace({"0": "ouro", "1": "prata"})
cluster

0    ouro
Name: Cluster, dtype: object

In [83]:
response02 = requests.request("POST", url , headers=headers, data=conteudo)
respostas02 = json.loads(response02.text.encode('utf8').decode())
respostas02

{'prediction': '0'}

In [84]:
inad = pd.Series(respostas02['prediction'], name="Inadimplência")
inad = inad.replace({"0": "paid", "1": "defaulted"})
inad

0    paid
Name: Inadimplência, dtype: object

In [85]:
clientes['Name'] = Name

In [86]:
pd.DataFrame(clientes.loc[:,"Name"])\
    .merge(right= cluster, left_index=True, right_index=True)\
    .merge(inad, left_index=True, right_index=True)

,Name,Cluster,Inadimplência
0,Teste,ouro,paid
